# Robot Navigation Kalman Filter Demonstration

The following notebook will step through a linear Kalman filter applied to raw data from a moving robot. The dataset consists of calibrated Inertial Measurement Unit (IMU) (accelerometer, gyroscope) and geomagnetic field (magnetometer) readings from an Android phone. The data was collected by positioning a smartphone flat as a path is traversed through a building.

The accelerometer measures the acceleration force along the x, y, z axes (including
gravitational acceleration) in metres per second 2 (m/s 2 ), the gyroscope measures the rate of rotation around the x, y, z axes in radians per second (rad/s), and the magnetometer measures the geomagnetic field strength along the x, y, and z axes in microtesla (µT ). All three sensors are calibrated but tend to exhibit significant noise and bias over time, with gyroscopes in particular often displaying unbounded drift. These noisy readings need to be corrected to derive the true position and orientation.

To improve the accuracy of indoor positioning using these sensor readings, we will be using a Kalman Filter, a recursive filter that uses state space techniques to find refined estimates for the state over time.

## Data set

The data was derived from the Kaggle [Indoor Location & Navigation](https://www.kaggle.com/competitions/indoor-location-navigation) competition, hosted by Microsoft Research. The raw data format consists of text files that contain an inital path configuration and sensor definition. 

In [ ]:
import os
import random
from datetime import datetime
from enum import Enum
from pathlib import Path

import numpy as np
from numpy.typing import NDArray


from estimators.sensors import (
    Accelerometer,
    AccelerometerUncalibrated,
    Gyroscope,
    GyroscopeUncalibrated,
    Magnetometer,
    MagnetometerUncalibrated,
    Measurement,
    RotationVector,
    Sensor,
    Waypoint,
)

data_dir = "data/indoor_robot/train/"

First we will load the sensor configuration

In [ ]:
config_file = "sensor_config.txt"

with open(sensor_config, "r", encoding="utf-8") as f:
    lines = f.readlines()

print(lines)

To parse this into a suite of sensors we will use the following function

In [ ]:
def create_sensor(config):
    params = {}
    # Parse config string into parameter list
    config = re.findall("([a-zA-Z]*):([a-zA-Z0-9 .]*)", config)
    for field, value in config:
        params[field] = value
    
    # Parse model name and type
    name = params.pop("name")
    id, type = name[: name.find(" ")], name[name.find(" ") + 1 :]
    params.update({"id": id, "type": type})

    # Grab sensor class from type name
    sensor = eval(type.replace(" ", ""))
    return sensor(**params)

In [ ]:
sensors = {}

for line in lines:
    if line.find("type:") != -1:
        sensor = create_sensor(line)
        if sensor:
            sensors[sensor.type.lower()] = sensor
            
print(sensors.keys())

Now we will load in all of the measurements and sources of truth from a randomly selected train file. The sources of truth for this data set is a series of waypoints that indicate the true (x, y) position of the robot over time. No other sources of truth were available and so the remaining state values will be evaluated in terms ofthe model uncertainty.

In [ ]:
# Randomly select test data set file
files = [f for f in os.listdir(fdir) if f != config_file]
fpath = fdir + random.choice(files)


with open(file_path, "r", encoding="utf-8") as f:
    lines = f.readlines()
    
print(lines[0:5])

To parse the measurements and load them into our sensors, we will use the following function

In [ ]:
def process_measurement(reading: str):
    data = reading.rstrip("\n").split("\t")

    t = datetime.fromtimestamp(int(data[0]) / 1000.0)
    measurement = data[2:]
    
    # Convert sensor reading definition into a sensor type 
    reading_type = data[1].lstrip("TYPE_")
    sensor_type = (
        reading_type.replace("MAGNETIC_FIELD", "MAGNETOMETER").lower().replace("_", " ")
    )
    return (
        t,
        sensor_type,
        measurement,
    )
    
waypoints = {}
time_ = set()

for line in lines:
    if line.startswith("#"):
        continue
    else:
        t, sensor_type, measurement = process_measurement(line)
        if sensor_type == "waypoint":
            waypoints[t] = Waypoint(t, *measurement)
        elif sensor := sensors.get(sensor_type):
            sensor.add_reading(t, *measurement)
        time_.add(t)
    
time_ = sorted(list(time_))
print(f"{len(time_)} sensor readings collected between {time_[0]} and {time_[-1]}")

Now, given our full data set, we can then begin defining our Kalman filter's state space and transition model

## Kalman filter definition
### State vector

The model will track the position (x, y, z), velocity, ( ẋ, ẏ, ż), acceleration ( ẍ, ÿ, z̈), as well as the pose (ψ, θ, φ), rotation rate ( ψ̇, θ̇, φ̇) and the gyroscope bias (ψ ε , θ ε , φ ε ). These values fully describe the position and orientation of the sensor over time. This gives us a state vector at time t of:

$$
X_t =  \left[
\begin{array}
        x_t  \\
        y_t \\
        z_t   \\
        \dot x_t  \\ 
        \dot y_t    \\
        \dot z_t   \\
        \ddot x_t   \\
        \ddot y_t  \\
        \ddot z_t  \\
        \psi_t  \\
        \theta_t  \\
        \phi_t   \\
        \dot \psi_t  \\ 
        \dot \theta_t    \\
        \dot \phi_t   \\
        \psi_{\epsilon, t}  \\ 
        \theta_{\epsilon, t}    \\
        \phi_{\epsilon, t}   \\
\end{array} \right]
$$

### State transition matrix

The model assume the acceleration of the robot over time to be linear, and uses this to derive estimates for the position and velocity of the robot, modelled by the kinematic equations:

$$
\left\{   \begin{array}{c} 
         x_t = x_{t -1} + \Delta t \dot x_{t - 1} + \frac{\Delta t ^2}{2} \ddot x_{t - 1} \\
         y_t = y_{t -1} + \Delta t \dot y_{t - 1} + \frac{\Delta t ^2}{2} \ddot y_{t - 1} \\
         z_t = z_{t -1} + \Delta t \dot z_{t - 1} + \frac{\Delta t ^2}{2} \ddot z_{t - 1} \\
         \dot x_t = \dot x_{t -1} + \Delta t \ddot x_{t - 1}  \\
         \dot y_t = \dot y_{t -1} + \Delta t \ddot y_{t - 1}  \\
         \dot z_t = \dot z_{t -1} + \Delta t \ddot z_{t - 1}  \\
         \ddot x_t = \ddot x_{t - 1}  \\
         \ddot y_t = \ddot y_{t - 1}  \\
         \ddot z_t = \ddot z_{t - 1}  \\ 
\end{array}  \right.  
$$

The gyroscope data can be modelled as:
\begin{equation}
    \omega_t = \hat \omega_t + \omega_{\epsilon, t} + n_g
\end{equation}

Where $\omega$ is the measured angular velocity, $\hat\omega$ is true orientation, $\omega_{\epsilon, t}$ is the gyroscope bias and $n_g$ is white Gaussian noise. 

Further assuming a linear angular rate of change, the measured angular velocity can be further written as:

\begin{equation}
    \omega_t = \omega_{t-1} + \Delta t \dot \omega_{t-1}
\end{equation}

We can therefore derive a representation of the current angular velocity as a function of the previous rate and bias:  

\begin{eqnarray}
    \omega_t & = & \omega_{t-1} + \Delta t \dot \omega_{t-1} \\
    & = & \omega_{t-1} + \Delta t \dot \omega_{t-1} - \omega_{\epsilon, t-1} - n_g \\
\end{eqnarray}


The state representation of orientation, rotation rate and gyroscope biases are therefore derived from the equations:

\begin{equation}
    \left\{   \begin{array}{c} 
         \psi_t = \psi_{t -1} + \Delta t \dot \psi_{t - 1} -  \psi_{\epsilon, t - 1} \\
         \theta_t = \theta_{t -1} + \Delta t \dot \theta_{t - 1} -  \theta_{\epsilon, t - 1} \\
         \phi_t = \phi_{t -1} + \Delta t \dot \phi_{t - 1} - \phi_{\epsilon, t - 1} \\
         \dot \psi_t = \dot \psi_{t -1}   \\
         \dot \theta_t = \dot \theta_{t -1}  \\
         \dot \phi_t = \dot \phi_{t -1}  \\
         \psi_{\epsilon, t} = \psi_{\epsilon, t - 1}  \\
         \theta_{\epsilon, t} = \theta_{\epsilon, t - 1}  \\
         \phi_{\epsilon, t} = \phi_{\epsilon, t - 1}  \\ 
    \end{array}  \right.  
\end{equation} 

### Measurements

For this model, linear acceleration, pose and rotation rate will be observed, represented by the matrix:
$$Y_t = \left[ \begin{array}
        \ddot x_t   \\
        \ddot y_t   \\
        \ddot z_t  \\
        \psi_t  \\
        \theta_t  \\
        \phi_t   \\
        \dot \psi_t  \\ 
        \dot \theta_t    \\
        \dot \phi_t   \\
\end{array} \right]
$$
However, these measurements are not available from the raw sensor readings, and must be derived.